In [1]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession

# from pyspark import SparkConf, SparkContext
from datetime import datetime, date, timedelta
from dateutil import relativedelta
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.functions import to_timestamp, to_date
from pyspark.sql import functions as F
from pyspark.sql.functions import collect_list, collect_set, concat, first, array_distinct, col, size, expr
import random

In [4]:
# import os

# #Manually doing it as there are some discrepancies with the Jupyter notebook server inheriting environment variables from the system \
# #or terminal settings 

# # Set JAVA_HOME
# os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home'

# # Adjust PATH to include JAVA_HOME/bin
# os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

# print("JAVA_HOME set to:", os.environ['JAVA_HOME'])
# print("PATH set to:", os.environ['PATH'])


'''
Solution: This can be fixed by directly adding the JAVA_HOME PATH in shell session files like .zshrc or .bash_profile
'''

'\nSolution: This can be fixed by directly adding the JAVA_HOME PATH in shell session files like .zshrc or .bash_profile\n'

In [3]:
# Initialize a SparkSession
spark = SparkSession.builder \
    .appName("Flight Data Analysis in Spark") \
    .getOrCreate()

24/04/23 19:33:50 WARN Utils: Your hostname, Rishikesans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.91 instead (on interface en0)
24/04/23 19:33:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/23 19:33:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Read the csv file 
flightData2015 = spark \
.read \
.option("inferSchema", "true")\
.option("header", "true")\
.csv("./2015-summary.csv")

In [6]:
type(flightData2015)

pyspark.sql.dataframe.DataFrame

In [20]:
# Display the information
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [8]:
#Count transformation with the lineage of steps that Spark will take during action 
flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#19 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#19 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=33]
      +- FileScan csv [DEST_COUNTRY_NAME#17,ORIGIN_COUNTRY_NAME#18,count#19] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/rishikesanravichandran/Documents/MSBA/Subjects/Spring/Big ..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [15]:
spark.conf.set("spark.sql.shuffle.partitions", "10")
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

<br><br>
# Dataframes and SQL <br><br>

In [16]:
#Create dataframe into a table or view
flightData2015.createOrReplaceTempView("flight_data_2015")

In [17]:
#SQL vs Dataframe in Spark

sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()
sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#17, 10), ENSURE_REQUIREMENTS, [plan_id=91]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/rishikesanravichandran/Documents/MSBA/Subjects/Spring/Big ..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#17, 10), ENSURE_REQUIREMENTS, [plan_id=104]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#17] Batched: false, DataFilters: [], Format: CSV, Location: InM

In [27]:
#Maximum flight count via SQL

spark.sql("SELECT max(count) from flight_data_2015").take(1)

[Row(max(count)=370002)]

In [26]:
#Maximum flight count via dataframe

from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [35]:
# Top 5 destination countries in the data using SQL

maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [36]:
# Top 5 destination countries in the data using dataframe

from pyspark.sql.functions import desc
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.show()


+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [38]:
# Explain or lineage of the previous query 

flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#296L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#17,destination_total#296L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[sum(count#19)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#17, 10), ENSURE_REQUIREMENTS, [plan_id=654]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#17], functions=[partial_sum(count#19)])
            +- FileScan csv [DEST_COUNTRY_NAME#17,count#19] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/rishikesanravichandran/Documents/MSBA/Subjects/Spring/Big ..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


